# Lista 5
#### Michał Ilski 250079

### Zadanie 1.

In [4]:
using SparseArrays

In [22]:
mutable struct CustomMatrix
    n::Int64
    l::Int64
    v::Int64
    A_blocks::Array{Array{Float64,2},1}
    B_blocks::Array{Array{Float64,2},1}
    C_blocks::Array{Array{Float64,2},1}
    function CustomMatrix(n::Int64, l::Int64)
        v = n/l
        A_blocks = [Array{Float64}(zeros(l,l)) for idx in 1:n]
        B_blocks = [Array{Float64}(zeros(l,l)) for idx in 1:n-1]
        C_blocks = [Array{Float64}(zeros(l,l)) for idx in 1:n-1]
        return new(n, l, v, A_blocks, B_blocks, C_blocks)
    end
end

In [23]:
function set_value(c::CustomMatrix, i::Int64, j::Int64, value::Float64)
    l = c.l
    k = ((i-1)÷l)*l
    if j in k+1:k+l
        c.A_blocks[(i÷l)+1][((i-1)%l+1), ((j-1)%l+1)] = value
    elseif j in k-l+1:k
        c.B_blocks[(i÷l)+1][((i-1)%l+1), ((j-1)%l+1)] = value
    elseif j in k+l+1:k+2*l
        c.C_blocks[(i÷l)+1][((i-1)%l+1), ((j-1)%l+1)] = value
    end
end

set_value (generic function with 1 method)

In [24]:
function get_value(c::CustomMatrix, i::Int64, j::Int64)
    l = c.l
    k = ((i-1)÷l)*l
    if j in k+1:k+l
        return c.A_blocks[(i÷l)+1][((i-1)%l+1), ((j-1)%l+1)]
    elseif j in k-l+1:k
        return c.B_blocks[(i÷l)+1][((i-1)%l+1), ((j-1)%l+1)]
    elseif j in k+l+1:k+2*l
        return c.C_blocks[(i÷l)+1][((i-1)%l+1), ((j-1)%l+1)]
    end
    return 0.0
end

get_value (generic function with 1 method)

#### Funkcja czytająca macierz z pliku  
Funkcja read_matrix na wejściu przyjmuje ścieżkę, w którym znajduje się plik z macierzą oraz nazwę pliku. W funkcji w pierwszej kolejności czytane są wartości $n$ (rozmiar macierzy) oraz $l$ (rozmiar bloków w macierzy głównej). Następnie każda kolejna linia to $i$ (pierwszy indeks macierzy), $j$ (drugi indeks macierzy), $A[i,j]$. Komórki macierzy, dla których nie zostały podane wartości w pliku przyjmują wartość $0$. Zwracana jest wczytana macierz, $n$ oraz $l$.

In [26]:
function read_matrix(folder::String,filename::String)
    path = "$folder/$filename"
    
    open(path, "r") do io
        n, l = (parse(Int64, x) for x in split(readline(io), " "))
        #matrix = spzeros(Float64, n, n)
        matrix = CustomMatrix(n,l)
        while !eof(io)
            line = split(readline(io), " ")
            i,j,value = (parse(Int64,line[1]), parse(Int64,line[2]), parse(Float64,line[3]))
            #matrix[i,j] = value
            set_value(matrix, i, j ,value)
        end
        
        return matrix, n, l
    end    
end

read_matrix (generic function with 1 method)

#### Funkcja czytająca wektor z pliku
Wektor wczytywany jest podobnie jak wyżej. Tutaj jednak jedynymi danymi są długość wektora $n$ oraz jego $n$ wartości. Zwracany jest wektor wraz z wymiarem.

In [27]:
function read_vector(folder::String,filename::String)
    path = "$folder/$filename"
    
    open(path, "r") do io
        n = parse(Int64, readline(io))
        vector = Vector{Float64}(zeros(n))
        for i in 1:n
            vector[i] = parse(Float64, readline(io))
        end
        
        return vector, n
    end    
end

read_vector (generic function with 1 method)

#### Funkcja generująca wektor $b$
Znając macierz $A$ oraz wiedząc, że wektor $x = [1,...,1]^{T}$, możemy samodzielnie wyznaczyć postać wektora $b$. W tym celu wykonuję mnożenie $Ax = b$. W pętli iteruję po wszystkich rzędach, natomiast ze względu na specyficzną postać macierzy $A$, nie muszę iterować po wszystkich kolumnach. Będąc w $i$-tym rzędzie, sumuję wartości w zakresie od $max\{1,l\cdot\lceil\frac{i-l}{l}\rceil\}$ do $min\{n,l\cdot\lceil\frac{i-l}{l}\rceil+l\}$, co obejmuje wartości z bloków $A_x$ i $B_x$ oraz w przypadku, gdy $i+l \leq n$, do $b[i]$ dodaję jeszcze $A[i, i+l]$, czyli bloki $C_x$.

In [36]:
function generate_b_vector(A::CustomMatrix, n::Int64, l::Int64)
    b = Vector{Float64}(zeros(n))
    for i in 1:n
        b[i] = sum(get_value(A,i,Int64(j)) for j in max(1,l*ceil((i-l)/l)):min(n,l*ceil((i-l)/l)+l))
        if i+l <= n
            b[i] += get_value(A,i,i+l)
        end
    end
    return b
end

generate_b_vector (generic function with 2 methods)

#### Funkcja pomocnicza drukująca macierz
Funkcja nie drukuje prawdziwych wartości macierzy, a jedynie ich wartości bezwzględne w ograniczonej przez parametr $prec$ precyzji. Używana w celu sprawdzenia wyglądu macierzy po rozkładzie.

In [48]:
function print_matrix(A, n, prec)
    for i in 1:n
        for j in 1:n
            print(round(abs(get_value(A,i,j)), digits=prec), " ")
        end
        println()
    end
end

print_matrix (generic function with 1 method)

In [58]:
function gaussian_elimination(A::CustomMatrix, b::Vector{Float64}, n::Int64, l::Int64)
    for k in 1:(n-1)
        for i in (k+1):min(k+l, n)
            lik = get_value(A,i,k)/get_value(A,k,k) #A[i,k]/A[k,k]
            set_value(A,i,k,0.0) #A[i,k] = 0.0
            for j in k+1:min(k+l, n)
                temp = get_value(A,i,j)
                set_value(A,i,j,temp-lik*get_value(A,k,j)) #A[i,j] -= lik*A[k,j]
            end
            b[i] -= lik*b[k]
        end
    end       
end

gaussian_elimination (generic function with 2 methods)

In [59]:
function solve_using_gaussian_elimination(A::CustomMatrix, b::Vector{Float64}, n::Int64, l::Int64)
    x = Vector{Float64}(zeros(n))
    for k in n:-1:1
        matrix_sum = 0
        for j = k + 1:min(n, k + l)
            matrix_sum += get_value(A,k,j)*x[j] #A[k, j] * x[j]
        end
        x[k] = (b[k] - matrix_sum)/get_value(A,k,k) #(b[k] - matrix_sum)/A[k,k]
    end
    return x
end

solve_using_gaussian_elimination (generic function with 1 method)

In [65]:
function gaussian_elimination_choose_element(A::CustomMatrix, b::Vector{Float64}, n::Int64, l::Int64)
    order = Vector(1:n)
    for k in 1:(n-1)
        #finding max abs value in column
        current_max, max_id = abs(get_value(A,k,k)), k
        for i in (k+1):min(k+l, n)
            if abs(get_value(A,i,k)) > current_max
                current_max, max_id = abs(get_value(A,i,k)), i
            end
        end
        order[max_id], order[k] = order[k], order[max_id]
        
        for i in (k+1):min(k+l, n)
            lik = get_value(A,order[i],k)/get_value(A,order[k],k) #A[order[i],k]/A[order[k],k]
            set_value(A,order[i],k,0.0)
            #2l bo wybrany element moze byc maksymalnie o l w dol, czyli potem o 2 l w prawo sprawdzamy
            for j in k+1:min(k+2*l, n)
                #A[order[i],j] -= lik*A[order[k],j]
                temp = get_value(A,order[i],j)
                set_value(A,order[i],j,temp-lik*get_value(A,order[k],j))     
            end
            b[order[i]] -= lik*b[order[k]]
        end
    end
    return order
end

gaussian_elimination_choose_element (generic function with 1 method)

In [69]:
function solve_using_gaussian_elimination_choose_element(A::CustomMatrix, b::Vector{Float64}, order::Vector{Int64}, n::Int64, l::Int64)
    x = Vector{Float64}(zeros(n))
    for k = n:-1:1
        matrix_sum = 0.0
        for j = k + 1:min(k+2*l, n)
            matrix_sum += get_value(A,order[k],j)*x[j] #A[order[k], j] * x[j]
        end

        x[k] = (b[order[k]] - matrix_sum)/get_value(A,order[k],k) #(b[order[k]] - matrix_sum)/A[order[k], k]
    end

    return x
end

solve_using_gaussian_elimination_choose_element (generic function with 1 method)

In [70]:
A,n,l = read_matrix("Dane16_1_1/", "A.txt")
b,n = read_vector("Dane16_1_1/", "b.txt")
gaussian_elimination(A,b,n,l)
solve_using_gaussian_elimination(A,b,n,l)

16-element Array{Float64,1}:
 1.0000000000000007
 1.0000000000000007
 1.0000000000000007
 1.0000000000000013
 0.9999999999999997
 0.9999999999999996
 0.9999999999999996
 0.9999999999999994
 1.0000000000000002
 1.0
 1.0
 1.0000000000000002
 1.0000000000000002
 1.0000000000000002
 1.0
 1.0000000000000002

In [71]:
A,n,l = read_matrix("Dane16_1_1/", "A.txt")
b,n = read_vector("Dane16_1_1/", "b.txt")
order = gaussian_elimination_choose_element(A,b,n,l)
solve_using_gaussian_elimination_choose_element(A,b,order,n,l)

16-element Array{Float64,1}:
 1.0
 1.0000000000000002
 1.0
 1.0000000000000002
 0.9999999999999992
 0.9999999999999993
 0.9999999999999993
 0.9999999999999992
 1.0000000000000004
 1.0000000000000002
 1.0000000000000004
 1.0000000000000002
 0.9999999999999999
 0.9999999999999997
 0.9999999999999999
 0.9999999999999998

### Zadanie 2/3.

In [12]:
function lu(A::SparseMatrixCSC{Float64, Int64}, n::Int64, l::Int64)
    for k in 1:(n-1)
        for i in (k+1):min(k+l, n)
            lik = A[i,k]/A[k,k]
            A[i,k] = lik
            for j in k+1:min(k+l, n)
                A[i,j] -= lik*A[k,j]
            end
        end
    end       
end

lu (generic function with 1 method)

In [13]:
function lu_choose_element(A::SparseMatrixCSC{Float64, Int64}, n::Int64, l::Int64)
    order = Vector(1:n)
    for k in 1:(n-1)
        #finding max abs value in column
        current_max, max_id = abs(A[k,k]), k
        for i in (k+1):min(k+l, n)
            if abs(A[i,k]) > current_max
                current_max, max_id = abs(A[i,k]), i
            end
        end
        order[max_id], order[k] = order[k], order[max_id]
        
        for i in (k+1):min(k+l, n)
            lik = A[order[i],k]/A[order[k],k]
            A[order[i],k] = lik
            #2l bo wybrany element moze byc maksymalnie o l w dol, czyli potem o 2 l w prawo sprawdzamy
            for j in k+1:min(k+2*l, n)
                A[order[i],j] -= lik*A[order[k],j]
            end
        end
    end
    return order
end

lu_choose_element (generic function with 1 method)

In [15]:
b, n = read_vector("Dane16_1_1/", "b.txt")

([2.5766127043066267, 0.22262875140362723, 2.5407983784381454, -0.8059533890034357, -0.9215537121431373, -1.7842163071531725, 6.177975978720661, 0.058735241454084985, 3.57440194333687, 2.2664366406035943, -0.6564884305649747, -0.1882688976733593, 0.4672075394498107, 4.3913570168563005, 1.32735255307743, -2.263440204814917], 16)

In [16]:
print_matrix(A,n,1)

4.6 1.2 1.3 2.2 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
6.0 2.6 3.9 0.3 0.0 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
1.2 6.8 3.3 2.3 0.0 0.0 0.2 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
1.5 7.6 0.5 5.7 0.0 0.0 0.0 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 2.5 4.2 3.7 1.9 0.2 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.1 4.9 0.2 1.5 4.0 0.0 0.3 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.2 3.2 6.9 1.2 5.3 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.2 4.9 6.5 1.3 2.5 0.0 0.0 0.0 0.3 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.5 7.8 2.1 1.7 0.1 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.7 1.6 4.3 3.2 0.0 0.2 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2 5.6 1.5 4.6 0.5 0.0 0.0 0.1 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2.3 6.6 2.6 5.8 0.0 0.0 0.0 0.2 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.3 6.5 3.3 3.2 0.2 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 6.7 1.4 2.3 6.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2 0.0 2.9 1.8 2.3 
0.0 0.0 0.0 0.0 0.0 0.0 0

In [19]:
function solve_lu(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, n::Int64, l::Int64)
    
    y = Vector{Float64}(zeros(n))
    for i in 1:n
        matrix_sum = 0.0
        for j in max(1, Int64(l * floor((i - 1) / l))):i-1
            matrix_sum += A[i,j]*y[j]
        end
        y[i] = b[i] - matrix_sum
    end
    
    x = Vector{Float64}(zeros(n))
    
    for k = n:-1:1
        matrix_sum = Float64(0.0)
        for j = k+1:min(n, k+l)
            matrix_sum += A[k,j] * x[j]
        end
        x[k] = (y[k] - matrix_sum)/A[k, k]
    end

    return x
end
    
        

solve_lu (generic function with 1 method)

In [67]:
function solve_lu_choose_element(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, order::Vector{Int64}, n::Int64, l::Int64)
    y = Vector{Float64}(zeros(n))
    for i in 1:n
        matrix_sum = 0.0
        for j in max(1, Int64(l * floor((i - 1) / l))):i-1
            matrix_sum += A[order[i],j]*y[j]
        end
        y[i] = b[order[i]] - matrix_sum
    end
    
    x = Vector{Float64}(zeros(n))
    
    for k = n:-1:1
        matrix_sum = Float64(0.0)
        for j = k+1:min(n, k+l)
            matrix_sum += A[order[k],j] * x[j]
        end
        x[k] = (y[k] - matrix_sum)/A[order[k], k]
    end

    return x
end

solve_lu_choose_element (generic function with 1 method)

In [69]:
A,n,l = read_matrix("Dane50000_1_1/", "A.txt")
b = generate_b_vector(A,n,l)
order = lu_choose_element(A, n, l)
x = solve_lu_choose_element(A,b,order,n,l)

50000-element Array{Float64,1}:
 1.0047788339368418
 0.9783314135064629
 1.0109912833604204
 0.9992261989552098
 1.0048335850451564
 1.0034338462186208
 1.003501903139483
 1.0017845920605484
 0.9485922235198623
 0.9807072699127508
 1.0028769889129403
 1.0014067433496663
 0.9829998639926145
 ⋮
 1.0111563412952056
 0.8782430551406178
 1.0022463674790725
 0.9958275968421209
 1.0327104950339703
 1.0597602534541528
 1.0
 1.0000000000000002
 1.0
 1.0
 1.0
 1.0